## 교차검증

In [30]:
### 08_wine.csv 파일 불러들이시고 출력까지

In [31]:
import pandas as pd
import numpy as np

In [32]:
file_path = './data/08_wine.csv'
wine = pd.read_csv(file_path)
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [33]:
### 독립변수와 종속변수 데이터 추출하기
# 독립변수명 : data
# 종속변수명 : target

In [34]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(data.shape, target.shape)

(6497, 3) (6497,)


## 교차검증 : train_test_split() 방식

In [35]:
### 훈련데이터 : 검증데이터 : 테스트데이터로 쪼개기
# - 보통 6 :2 :2로 쪼갭니다.
# - 보통 많이 사용하는 방식..

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
train_input, test_input, train_target, test_target = \
train_test_split(data, target, test_size=0.2, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [38]:
### 2. 훈련데이터를 이용해서 검증데이터 생성하기

In [39]:
sub_input, val_input, sub_target, val_target = \
train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [40]:
from sklearn.tree import DecisionTreeClassifier

In [48]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [49]:
print(dt.score(test_input, test_target))

0.8569230769230769


## 교차검증 시작...

In [ ]:
### 훈련데이터와 검증데이터를 구분할 필요없이
# - 내부적으로 구분해서 사용..
# - 내부적으로 구분에 사용하는 데이터는 훈련(train) 데이터
# - 데이터 준비는 기존처럼 훈련과 테스트 데이터만 준비..
# - 검증데이터는 별도로 준비 안해도 됨..

In [ ]:
### 교차검증에 사용되는 주요 키워드
# 3-폴드(fold) 교차검증
# : 훈련데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
# : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
# : k-폴드 교차검증 또는 k-겹 교차검증이라고도 합니다.

# 보통 5-폴드 교차검증 또는 10-폴드 교차검증을 사용합니다.
# : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능

In [46]:
### 사용하는 데이터 다시한번 확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [56]:
### 교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate

# dt : 결정트리 훈련모델(다른 모델을 사용한 경우 해당 모델)
# 두번째값 : 훈련데이터(fold에서 훈련데이터 쪼갤때 사용)
# 세번째값 : 검증데이터(fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(dt, train_input, train_target)
# print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('test_score의 평균 = ', scores['test_score'].mean())

test_score의 평균 =  0.855300214703487


In [64]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

# n_splits : fold 갯수, shuffle : 섞기
splitter = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, \
                       cv = splitter)

# print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('test_score의 평균 = ', scores['test_score'].mean())

test_score의 평균 =  0.8603073953073954


## 하이퍼파라미터 튜닝(AutoML)

In [ ]:
### 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
# - 이런 개념을 AutoML이라고 합니다.
# - 사용패키지 : sklearn.model_selection
# - 사용클래스 : GridSearchCV
# - 그리드 서치라고도 합니다.

## 결정트리의 max_depth(트리의 깊이) 값 찾기
##### - 그리드서치(GridSearchCV) 객체(모델) 사용

In [68]:
from sklearn.model_selection import GridSearchCV

# 찾을 하이퍼파라미터는 딕셔너리로
# 실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {'max_depth' : range(5, 20, 1)}

# 객체(모델) 생성
# - 첫번째 값 : 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼파라메터 값들
# - 세번째 값 : CPU 코어 갯수 (-1은 모든 코어 사용, 병렬 처리)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, n_jobs = -1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [69]:
# 그리드서치의 결과..
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요 : 훈련모델이 생성되어 반환됨
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [73]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9003271117952665

In [74]:
gs.score(train_input, train_target)

0.9003271117952665

In [75]:
gs.best_params_

{'max_depth': 8}

In [78]:
### 딕셔너리의 key값만 조회하기
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [80]:
### params 값 확인하기
gs.cv_results_['params']

[{'max_depth': 5},
 {'max_depth': 6},
 {'max_depth': 7},
 {'max_depth': 8},
 {'max_depth': 9},
 {'max_depth': 10},
 {'max_depth': 11},
 {'max_depth': 12},
 {'max_depth': 13},
 {'max_depth': 14},
 {'max_depth': 15},
 {'max_depth': 16},
 {'max_depth': 17},
 {'max_depth': 18},
 {'max_depth': 19}]

In [81]:
### 1폴드 조회하기
gs.cv_results_['split0_test_score']

array([0.84711538, 0.84807692, 0.85769231, 0.85288462, 0.85769231,
       0.84519231, 0.85865385, 0.86730769, 0.86538462, 0.86826923,
       0.86730769, 0.86442308, 0.86346154, 0.86634615, 0.87211538])

In [84]:
### 훈련모델 최종 검증하기
dt.score(test_input, test_target)

0.8584615384615385

In [108]:
df = pd.DataFrame(gs.cv_results_['params'])
df_columns = ['max_depth']

for n in range(5):
    df = pd.concat([df, pd.DataFrame(gs.cv_results_['split{}_test_score'.format(n)])], axis = 1)
    df_columns.append('split{}_test_score'.format(n))

df.columns = df_columns

df

,max_depth,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,5,0.847115,0.863462,0.880654,0.836381,0.861405
1,6,0.848077,0.854808,0.873917,0.843118,0.859480
2,7,0.857692,0.851923,0.875842,0.843118,0.847931
3,8,0.852885,0.854808,0.878730,0.846968,0.865255
4,9,0.857692,0.859615,0.871030,0.842156,0.854668
5,10,0.845192,0.847115,0.876805,0.846968,0.852743
6,11,0.858654,0.850962,0.880654,0.852743,0.848893
7,12,0.867308,0.848077,0.882579,0.848893,0.848893
8,13,0.865385,0.853846,0.873917,0.853705,0.839269
9,14,0.868269,0.851923,0.870067,0.858518,0.841193


## 찾을 하이퍼파라미터 여러개 사용하기

In [110]:
# min_impurity_decrease : 결정트리 노드 분리시 불순도 감소 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), \
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

### 그리드서치 훈련시키기
# 찾을 파라미터의 범위 별로 교차검증 진행 : 엄청 많이 수행됨
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [111]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [112]:
gs.score(train_input, train_target)

0.892053107562055

In [114]:
gs.score(test_input, test_target)

0.8615384615384616

In [115]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [121]:
### 10-폴드 지정하기
splitter = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

# min_impurity_decrease : 결정트리 노드 분리시 불순도 감소 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), \
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

### 그리드서치 훈련시키기
# 찾을 파라미터의 범위 별로 교차검증 진행 : 엄청 많이 수행됨
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, cv=splitter, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [120]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'split10_test_score', 'split11_test_score', 'split12_test_score', 'split13_test_score', 'split14_test_score', 'split15_test_score', 'split16_test_score', 'split17_test_score', 'split18_test_score', 'split19_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [122]:
dt = gs.best_estimator_

In [123]:
dt.score(train_input, train_target)

0.9230325187608236

In [124]:
dt.score(test_input, test_target)

0.8692307692307693

## 랜덤서치(Random Search)

In [ ]:
### 랜덤서치 사용시 아래 조건 만족해야 합니다.
# - 하이퍼파라미터값이 수치인 것들만 사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸리는 경우
# - 매개변수를 샘플링할때 확률분포객체를 전달..

# 사용 모듈 : uniform(실수값), randint(정수값)

In [125]:
from scipy.stats import uniform, randint

In [127]:
### 범위 설정
rgen = randint(0, 10)
rgen

### 범위 내에서 임의값 출력하기(확률적으로 고르게 뽑아냅니다.)
print(rgen.rvs(100))

[9 8 7 2 0 0 9 9 6 7 9 0 7 4 7 0 3 6 9 7 6 9 0 4 3 3 6 9 6 2 4 5 1 3 3 7 8
 8 3 2 5 0 3 7 2 3 3 5 9 1 1 6 3 4 4 4 0 6 2 5 5 5 8 0 7 7 2 7 7 5 6 8 5 5
 3 4 5 0 2 0 8 2 9 6 8 9 1 8 3 0 5 2 6 6 2 1 9 6 1 8]


In [128]:
# 실수값
# 범위 설정
ugen = uniform(0, 1)

# 임의값 50개 조회
print(ugen.rvs(50))

[0.30183906 0.18255695 0.34557614 0.69331658 0.41971585 0.10674811
 0.77107884 0.77526694 0.5748313  0.61795156 0.13245665 0.67500338
 0.09146813 0.95144444 0.50148641 0.88831996 0.92031484 0.06698014
 0.65162661 0.09735919 0.91265676 0.05539478 0.51869263 0.3848559
 0.0025266  0.22402454 0.55242671 0.97409426 0.5242336  0.19550049
 0.90801342 0.06117695 0.58063368 0.34930652 0.60583215 0.2424206
 0.7462982  0.38351623 0.41825323 0.17653856 0.7823008  0.69721851
 0.74412842 0.99577962 0.79539608 0.75613206 0.8291998  0.41889986
 0.04649113 0.76827206]


In [132]:
### 하이퍼파라미터 찾을 매개변수 정의하기
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델 사용, 훈련반복 회수 100회, 코어 전체, 시드 42

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params, n_iter=100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026DCA087910>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026DCA42FAC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026DC9F734F0>},
                   random_state=42)

In [133]:
gs.best_params_

{'max_depth': 29,
 'min_impurity_decrease': 0.000437615171403628,
 'min_samples_split': 16}

In [135]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [137]:
### 훈련 100회 중에 가장 큰값이 최종 평가점수
max(gs.cv_results_['mean_test_score'])

0.8695409787517583

In [138]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.8623076923076923